In [29]:

from IPython.core.display import display, HTML
toggle_code_str = '''
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Sloution"></form>
'''

toggle_code_prepare_str = '''
    <script>
    function code_toggle() {
        if ($('div.cell.code_cell.rendered.selected div.input').css('display')!='none'){
            $('div.cell.code_cell.rendered.selected div.input').hide();
        } else {
            $('div.cell.code_cell.rendered.selected div.input').show();
        }
    }
    </script>

'''

display(HTML(toggle_code_prepare_str + toggle_code_str))

def hide_sloution():
    display(HTML(toggle_code_str))


In [31]:
hide_sloution()

import pandas as pd
from google.oauth2 import service_account

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import ztest
from statsmodels.stats import weightstats
from datetime import datetime
import statsmodels.stats.weightstats as ws
import math 

import plotly.io as pio
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

pio.renderers.default='notebook' # для отображения графиков плотли в nbviewer
init_notebook_mode(connected = True) # для работы с плотли оффлайн
cf.go_offline()


def categ_sql():
    credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')
    project_id = 'spry-compound-139714' 
   
    query = '''
    select 
        *
    from `spry-compound-139714.SNTO_lyapakhina.head_catalog_events11`
    '''
    ct = pd.read_gbq(query, project_id=project_id, credentials=credentials)
    return ct



def categories1(ct):
    categ = ct[(ct['eventAction']=='header_nav_catalog_click')][['hitId', 'eventLabel']]
    categ = categ[categ['eventLabel'].str.contains("/") == True]
    categ['eventLabel'] = categ['eventLabel'].apply(lambda x: x.lower())
    categ = categ[(categ['eventLabel'].str.contains("каталог товаров") == True) | (categ['eventLabel'].str.contains("каталог") == True)]
    categ['one'] = categ['eventLabel'].apply(lambda x: x[x.index('/', 0)+1:])
    categ['one'] = categ['one'].apply(lambda x: x[:x.index('/', 0)] if ("/" in x) == True else x)
    categ['last'] = categ['eventLabel'].apply(lambda x: x[x.rindex('/', 0)+1:])
    categ = categ[['one', 'last', 'hitId']]
    categ = categ.groupby(['one', 'last'])['hitId'].count().reset_index().rename(columns = {'hitId': 'count_sum'})
   
    import plotly.express as px
    import numpy as np


    df = categ
    fig = px.treemap(df, path=[px.Constant("all"), 'one', 'last'], values='count_sum',
    #                   color='count_sum', 
    #                  hover_data=['count_sum'],
                      color_continuous_scale='RdBu',
                     color_continuous_midpoint=np.average(df['count_sum'], weights=df['count_sum'])
                    )
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    return fig.show()

  
    
    
    

In [32]:
hide_sloution()

ct = categ_sql()
categories1(ct)
